In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords

In [2]:
dataset = pd.read_csv("reviews.csv",sep="\t")

C:\Users\Jyoti\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
tfidf_vectorizer = TfidfVectorizer()

In [5]:
dataset.reset_index()
dataset.set_index("review_id")

,marketplace,customer_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,timestamp,IP Address
review_id,,,,,,,,,,,,,,,,
R8MEA6IGAHO0B,US,20422322,B00MC4CED8,82850235,BlackVue DR600GW-PMP,Mobile_Electronics,5,0.0,0.0,N,Y,Very Happy!,"As advertised. Everything works perfectly, I'm...",2015-08-31,1.440988e+09,193.93.167.87
R31LOQ8JGLPRLK,US,40835037,B00OQMFG1Q,82850235,GENSSI GSM / GPS Two Way Smart Phone Car Alarm...,Mobile_Electronics,5,0.0,1.0,N,Y,five star,it's great,2015-08-31,1.441002e+09,193.93.167.87
R2Y0MM9YE6OP3P,US,51469641,B00QERR5CY,82850235,iXCC Multi pack Lightning cable,Mobile_Electronics,5,0.0,0.0,N,Y,great cables,These work great and fit my life proof case fo...,2015-08-31,1.440959e+09,193.93.167.87
RRB9C05HDOD4O,US,4332923,B00QUFTPV4,82850235,abcGoodefg® FBI Covert Acoustic Tube Earpiece ...,Mobile_Electronics,4,0.0,0.0,N,Y,Work very well,Work very well,2015-08-31,1.441015e+09,193.93.167.87
R26I2RI1GFV8QG,US,44855305,B0067XVNTG,563475445,Generic Car Dashboard Video Camera Vehicle Vid...,Mobile_Electronics,2,0.0,0.0,N,Y,Cameras has battery issues,"Be careful with these products, I have bought ...",2015-08-31,1.440973e+09,205.10.168.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R2CDRVDUKB5Z9P,US,38978459,B00005OTZQ,554527960,Royal SE 2800 Hand-Held Spot Cleaner,Mobile_Electronics,5,23.0,27.0,N,N,The Cat Barf is Gone!,"I've been looking for a while for the \\""purr\...",2002-04-03,1.017782e+09,201.30.21.211
R1DVLTZFXXOX9,US,51697602,B00005OTZQ,554527960,Royal SE 2800 Hand-Held Spot Cleaner,Mobile_Electronics,5,10.0,11.0,N,N,Well worth [it],We live in an apartment with hardwood floors a...,2002-03-05,1.015341e+09,216.59.233.79
R1GHKKZMHAHQC1,US,50891675,B00005OTZQ,554527960,Royal SE 2800 Hand-Held Spot Cleaner,Mobile_Electronics,5,13.0,13.0,N,N,Its a Time saver!!!,I received this for a birthday present and Its...,2002-02-19,1.014068e+09,209.193.182.91


In [6]:
dataset.sort_values("timestamp",inplace=True)

In [7]:
def OnlyStopwords(str):
    words = nltk.word_tokenize(str)
    words = [word for word in words if word not in stopwords.words("english")]
    if(len(words)==0):
        return True
    return False

In [8]:
OnlyStopwords("to")

True

In [ ]:
from nltk.corpus import wordnet

remove_reviews = []
indices = []
for i in range(len(dataset)):
    
    reviews = [str(dataset["review_body"][i])]
    
    try:
        tfidf_vectorizer.fit_transform(reviews)
    except:
        # reviews with one word and with no dictionary meaning will be invalid
        # e.g- ["c","O.K."]
        remove_reviews.append(dataset["review_id"][i]) 
        continue
        
    Time = dataset["timestamp"][i]
    # timestamp of the review that will be compared
    
    for j in range(i+1,len(dataset)):
            
        indices.append(dataset["review_id"][j])
        
        if(dataset["timestamp"][j]-Time <= 1800):
            # reviews written in 30 min of intervals will be checked for same pattern
            reviews.append(str(dataset["review_body"][j]))
        else:
            break
    
    tfidf_matrix = tfidf_vectorizer.fit_transform(reviews)
    
    #creates TF-IDF Model
    tfidf_list = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix).tolist()
    # Creates matrix based on document similarity
         
    # To check similarity b/w 2 reviews
    i_appended = False
    for k in range(1,len(tfidf_list[0])):
        #print(tfidf_list[0][k],i+k)
        
        if(tfidf_list[0][k]>0.6):
            # 0.6 is defind for the simmilarity level
            
            remove_reviews.append(dataset["review_id"][i+k])
            # i+k is to get the review id of the review
            
            if(not i_appended):
                remove_reviews.append(dataset["review_id"][i]) 
                i_appended = True
 

In [ ]:
remove_reviews